In [1]:
import pandas as pd
from hashlib import sha1

In [3]:
papers = pd.read_csv("neo4j/papers.csv")

In [4]:
papers.shape

(100, 21)

In [5]:
papers.head(3)

,id:ID(paper),authors,affiliations,email,body_text,title,abstract,doi,source_x,pmcid,...,license,publish_time,venue,mag_id,who_covidence_id,arxiv_id,s2_id,url,publication_year,:LABEL
0,1fe645c8d05d960dd85d08eaaa5574100db591c8,Iresha N Ambepitiya Wickramasinghe; M Hélène V...,;,;,"{""Introduction"": ""Infectious bronchitis virus ...",Protein Histochemistry Using Coronaviral Spike...,Protein histochemistry is a tissue-based techn...,10.1007/978-1-4939-2438-7_14,Medline; PMC,PMC7121471,...,no-cc,2014-12-18,Coronaviruses,NaN,NaN,NaN,235989.0,https://www.ncbi.nlm.nih.gov/pubmed/25720479/;...,2014.0,paper
1,579fdd7af308a5a9b022f5a38c69b6d1fd3c6224,Manuela Bonizzoli; Rosaria Arvia; · Simona Di ...,; ; ; ; ; ;,; ; ; ; ; ;,"{""Introduction"": ""Human herpes simplex virus 1...",Human herpesviruses respiratory infections in ...,Acute respiratory distress syndrome (ARDS) is ...,10.1007/s00430-016-0456-z,Medline; PMC,PMC7086591,...,no-cc,2016-05-02,Med Microbiol Immunol,NaN,NaN,NaN,2707991.0,https://www.ncbi.nlm.nih.gov/pubmed/27138606/;...,2016.0,paper
2,14241795a7c1b25b0db12f42473176d5af9a2eca,Hai-Ming Wang; Tian-Xin Liu; · Tong-Yun Wang; ...,; ; ; ; ; ; ;,; ; ; ; ; ; ;,"{""Introduction"": ""Porcine reproductive and res...",Isobavachalcone inhibits post-entry stages of ...,Porcine reproductive and respiratory syndrome ...,10.1007/s00705-018-3755-4,Medline; PMC,PMC7086980,...,no-cc,2018-02-06,Arch Virol,NaN,NaN,NaN,4583634.0,https://doi.org/10.1007/s00705-018-3755-4; htt...,2018.0,paper


In [9]:
venue = papers.venue.unique().tolist()
vdata = []
for v in venue:
    if not pd.isna(v):
        vdata.append([sha1(v.encode()).hexdigest(), v, "journal"])
venue = pd.DataFrame(data=vdata, columns=["id:ID(journal)", "name", ":LABEL"])
venue.head()

,id:ID(journal),name,:LABEL
0,6092eb62e87154c96f6642969984df334731d3ab,Coronaviruses,journal
1,19ec223414598012c79e1257098d10a983aaa920,Med Microbiol Immunol,journal
2,e87cf289d88c1d63bec4f294a695fe092262be30,Arch Virol,journal
3,dc8769707eace55e9b51f588da024ed6e4e23127,Encyclopedia of Respiratory Medicine,journal
4,2751758e49d5ec776e9ee34f2d1e63d3ccdaf4ee,BMC Evol Biol,journal


In [10]:
venue.to_csv("neo4j/journals.csv", index=False)

In [11]:
jpdata = []
for i in range(len(papers)):
    v = papers.venue[i]
    if not pd.isna(v):
        jpdata.append([sha1(v.encode()).hexdigest(), papers["id:ID(paper)"][i], "has_paper"])

jp = pd.DataFrame(data=jpdata, columns=["journal_id:START_ID(journal)", "paper_id:END_ID(paper)", ":TYPE"])
jp.head()

,journal_id:START_ID(journal),paper_id:END_ID(paper),:TYPE
0,6092eb62e87154c96f6642969984df334731d3ab,1fe645c8d05d960dd85d08eaaa5574100db591c8,has_paper
1,19ec223414598012c79e1257098d10a983aaa920,579fdd7af308a5a9b022f5a38c69b6d1fd3c6224,has_paper
2,e87cf289d88c1d63bec4f294a695fe092262be30,14241795a7c1b25b0db12f42473176d5af9a2eca,has_paper
3,dc8769707eace55e9b51f588da024ed6e4e23127,d7cb7c135b8d3727e626d35b7b1ec92f2400e78b,has_paper
4,2751758e49d5ec776e9ee34f2d1e63d3ccdaf4ee,479ade65b7f5219559c4d8cc48c1c2f90bda977f,has_paper


In [12]:
jp.to_csv("neo4j/journal_paper_rel.csv", index=False)

In [17]:
authors = []

for i in range(len(papers)):
    if not pd.isna(papers.authors[i]):
        authors += papers.authors[i].split("; ")

authors = list(set(authors))
len(authors)

594

In [18]:
adata = []
for a in authors:
    if not pd.isna(a):
        adata.append([sha1(a.encode()).hexdigest(), a, "author"])
        
authors = pd.DataFrame(data=adata, columns=["id:ID(author)", "name", ":LABEL"])
authors.head()

,id:ID(author),name,:LABEL
0,7f334d4fa026694142a6ddc93232122162af9739,José María Barrios-López,author
1,a5fa698de58c39df886df566f65955657c9f8730,Rahmat Awang,author
2,d10f26ef3f5262c1ba2d60ccf03b8a4f0c023f14,Michał Nowakowski,author
3,836d016a8dd69e1e7c1e17e28868349a333f2c63,Gui-Xia Li,author
4,b56e6bc303475fd4fc46e752f2b685693b08d1e3,Marion E Weir,author


In [19]:
authors.to_csv("neo4j/authors.csv", index=False)

In [21]:
apdata = []
for i in range(len(papers)):
    a = papers.authors[i]
    if not pd.isna(a):
        a = papers.authors[i].split("; ")
        for auth in a:
            apdata.append([sha1(auth.encode()).hexdigest(), papers["id:ID(paper)"][i], "has_published"])

ap = pd.DataFrame(data=apdata, columns=["author_id:START_ID(author)", "paper_id:END_ID(paper)", ":TYPE"])
ap.head()

,author_id:START_ID(author),paper_id:END_ID(paper),:TYPE
0,799d3471dc131fdc25d248246adaa6efc753fc72,1fe645c8d05d960dd85d08eaaa5574100db591c8,has_published
1,5400c3bc7dde46fd671390a648659e4697ca9ec9,1fe645c8d05d960dd85d08eaaa5574100db591c8,has_published
2,bf1e7d9e12fa464e41a87b27f4dfbe1847da17e6,579fdd7af308a5a9b022f5a38c69b6d1fd3c6224,has_published
3,5bb5106a45d50ff39e7024117a1b87cd60d1fc79,579fdd7af308a5a9b022f5a38c69b6d1fd3c6224,has_published
4,75397622a3c9b5a707a7feb367f2afb5b14f6698,579fdd7af308a5a9b022f5a38c69b6d1fd3c6224,has_published


In [22]:
ap.shape

(595, 3)

In [23]:
ap.to_csv("neo4j/author_paper_rel.csv", index=False)